In [2]:
import uuid
from random import random

import pandas as pd
import sqlalchemy as sa
from sqlalchemy.orm import declarative_base, Session

## Организация БД в SQLAlchemy ORM ##

In [3]:
DB_PATH = 'sqlite:///model3.sqlite3'
Base = declarative_base()
ENGINE = sa.create_engine(DB_PATH)

In [4]:
'''
class Model_Base(Base):
    __tablename__ = 'Students'
    uid = sa.Column(sa.Integer(), nullable=False, unique=True, primary_key=True, autoincrement=True)
    date = sa.Column(sa.String())
    tg_id = sa.Column(sa.Text())
    time_zone = sa.Column(sa.String())
    tech_stack = sa.Column(sa.Text())
    occupation = sa.Column(sa.Text())
    role = sa.Column(sa.Text())
    project_role = sa.Column(sa.Text())
    weekload = sa.Column(sa.Text())
    course = sa.Column(sa.Text())
    course_time = sa.Column(sa.Text())
    notes = sa.Column(sa.Text())
    pl = sa.Column(sa.String())
    chat = sa.Column(sa.String())
    exited = sa.Column(sa.Text())
    prediction = sa.Column(sa.Float())
'''
class Model_Base(Base):
    __tablename__ = 'Students'
    uid = sa.Column(sa.Integer(), nullable=False, unique=True, primary_key=True, autoincrement=True)
    date = sa.Column(sa.String())
    id = sa.Column(sa.Text())
    utc = sa.Column(sa.String())
    steck = sa.Column(sa.Text())
    spec = sa.Column(sa.Text())
    role = sa.Column(sa.Text())
    role_in = sa.Column(sa.Text())
    hour_per_week = sa.Column(sa.Text())
    other_courses = sa.Column(sa.Text())
    time_of_studies = sa.Column(sa.Text())
    notes = sa.Column(sa.Text())
    language = sa.Column(sa.String())
    in_chat = sa.Column(sa.String())
    out = sa.Column(sa.Text())
    prediction = sa.Column(sa.Float())

In [5]:
Base.metadata.create_all(ENGINE)

## Чтение из файла ##

In [6]:
# ini_df = pd.read_csv('data/anketa_ini.csv')
# ini_df = pd.read_csv('data/anketa_ini.csv', sep=';', encoding='utf-8')
ini_df = pd.read_excel('data/anketa_ini.xlsx')

## Маппинг для переименования колонок из файла в базу ##

In [8]:
old_cols = [
    'Отметка времени',
    'Профиль телеграмм (в формате @username)',
    'Часовой пояс',
    'Стек технологий',
    'По какой специальности SF хотите заявиться?',
    'Роль',
    'В какой роли видите себя в проекте?',
    'Сколько часов в неделю готовы уделять проекту?',
    'Какие другие курсы закончили или находитесь в процессе обучения?',
    'Как долго учитесь на курсах?',
    'Notes',
    'ЯП',
    'Вступление в чат практики.',
    'Выбыл'
]
'''
new_cols = [
    'date',
    'tg_id',
    'time_zone',
    'tech_stack',
    'occupation',
    'role',
    'project_role',
    'weekload',
    'course',
    'course_time',
    'notes',
    'pl',
    'chat',
    'exited'
]
'''
new_cols = [
    'date',
    'id',
    'utc',
    'steck',
    'spec',
    'role',
    'role_in',
    'hour_per_week',
    'other_courses',
    'time_of_studies',
    'notes',
    'language',
    'in_chat',
    'out'
]

In [9]:
def map_names(old_list: list, new_list: list) -> dict:
    dict_ = dict()
    if len(old_list) == len(new_list):
        for old_name, new_name in zip(old_list, new_list):
            dict_[old_name] = new_name
    return dict_

ini_df.rename(columns=map_names(old_cols, new_cols), inplace=True)
ini_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             320 non-null    datetime64[ns]
 1   id               321 non-null    object        
 2   utc              322 non-null    object        
 3   steck            321 non-null    object        
 4   spec             322 non-null    object        
 5   role             322 non-null    object        
 6   role_in          322 non-null    object        
 7   hour_per_week    322 non-null    object        
 8   other_courses    322 non-null    object        
 9   time_of_studies  322 non-null    object        
 10  notes            99 non-null     object        
 11  language         205 non-null    object        
 12  in_chat          322 non-null    object        
 13  out              31 non-null     object        
dtypes: datetime64[ns](1), object(13)
memory us

## Преобразование **`time_stamp`** пока в текст ##

In [10]:
# ini_df.time_stamp = ini_df.time_stamp.apply(lambda x: str(x))
ini_df.date = ini_df.date.apply(lambda x: str(x))

## Заполнение отсутствующих tg-погонял ##

In [11]:
ini_df['id'] = ini_df['id'].fillna(value='@unknown')
ini_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             322 non-null    object
 1   id               322 non-null    object
 2   utc              322 non-null    object
 3   steck            321 non-null    object
 4   spec             322 non-null    object
 5   role             322 non-null    object
 6   role_in          322 non-null    object
 7   hour_per_week    322 non-null    object
 8   other_courses    322 non-null    object
 9   time_of_studies  322 non-null    object
 10  notes            99 non-null     object
 11  language         205 non-null    object
 12  in_chat          322 non-null    object
 13  out              31 non-null     object
dtypes: object(14)
memory usage: 35.3+ KB


## Запись нормального CSV-файла, с которым потом можно работать (sep='^' для уникальности) ##

In [12]:
ini_df.to_csv('data/anketa_new.csv', sep='^', encoding='utf-8', index=False)

## Добавка целевого признака **`prediction`** и забивка случайными значениями ##

In [13]:
ini_df['prediction'] = round(random(), 2)
ini_df['prediction'] = ini_df['prediction'].apply(lambda x: round(random(), 2))
display(ini_df.prediction)

0      0.88
1      0.73
2      0.71
3      0.17
4      0.83
       ... 
317    0.75
318    0.94
319    0.85
320    0.28
321    0.19
Name: prediction, Length: 322, dtype: float64

## Ручное изготовление БД и проверка ##

In [14]:
'''
with Session(ENGINE) as s_:
    for index_ in range(ini_df.shape[0]):
        new_line_ = Model_Base(date=ini_df.date.iloc[index_],
                               tg_id=ini_df.tg_id.iloc[index_],
                               time_zone=ini_df.time_zone.iloc[index_],
                               tech_stack=ini_df.tech_stack.iloc[index_],
                               occupation=ini_df.occupation.iloc[index_],
                               role=ini_df.role.iloc[index_],
                               project_role=ini_df.project_role.iloc[index_],
                               weekload=ini_df.weekload.iloc[index_],
                               course=ini_df.course.iloc[index_],
                               course_time=ini_df.course_time.iloc[index_],
                               notes=ini_df.notes.iloc[index_],
                               pl=ini_df.pl.iloc[index_],
                               chat=ini_df.chat.iloc[index_],
                               exited=ini_df.exited.iloc[index_],
                               prediction=ini_df.prediction.iloc[index_]
                              )
        s_.add(new_line_)
    s_.commit()
'''
with Session(ENGINE) as s_:
    for index_ in range(ini_df.shape[0]):
        new_line_ = Model_Base(date=ini_df.date.iloc[index_],
                               id=ini_df.id.iloc[index_],
                               utc=ini_df.utc.iloc[index_],
                               steck=ini_df.steck.iloc[index_],
                               spec=ini_df.spec.iloc[index_],
                               role=ini_df.role.iloc[index_],
                               role_in=ini_df.role_in.iloc[index_],
                               hour_per_week=ini_df.hour_per_week.iloc[index_],
                               other_courses=ini_df.other_courses.iloc[index_],
                               time_of_studies=ini_df.time_of_studies.iloc[index_],
                               notes=ini_df.notes.iloc[index_],
                               language=ini_df.language.iloc[index_],
                               in_chat=ini_df.in_chat.iloc[index_],
                               out=ini_df.out.iloc[index_],
                               prediction=ini_df.prediction.iloc[index_]
                              )
        s_.add(new_line_)
    s_.commit()

In [17]:
output_list = []
with Session(ENGINE) as t_:
    all_predictions = t_.query(Model_Base).all()
for student in all_predictions:
    output_list.append({'id': student.id,
                        'role_in': student.role_in,
                        'prediction': student.prediction
                      })
##### print(output_list)